In [13]:
import pandas as pd

df = pd.read_csv('archive/data.csv', encoding='latin-1')

In [14]:
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom


In [35]:
import datetime

query_df = pd.DataFrame({
    'CustomerID': [17850, 13047, 12583],
    'InvoiceDate': [pd.to_datetime(datetime.date(2010, 12, 15))] * 3
})

In [57]:
# get historical features
def get_historical_data(df):
    df = df[~df.CustomerID.isna()]
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']).dt.floor('d')
    df_grouped = df.groupby(['InvoiceDate', 'CustomerID'])['UnitPrice'].agg(['count', 'sum']).reset_index()
    
    return df_grouped

In [82]:
df_grouped = get_historical_data(df)

In [83]:
merged_df = query_df.merge(df_grouped, how='left', on=['CustomerID'])

In [84]:
merged_df['InvoiceDate_lagged'] = merged_df['InvoiceDate_x'] - pd.DateOffset(days=30)

In [85]:
merged_df = merged_df[
    (merged_df.InvoiceDate_y < merged_df.InvoiceDate_x) &\
    (merged_df.InvoiceDate_y >= merged_df.InvoiceDate_lagged)
]

In [86]:
merged_df.groupby(['CustomerID', 'InvoiceDate_x'])[['count', 'sum']].mean()

,,count,sum
CustomerID,InvoiceDate_x,,
12583,2010-12-15,20.0,55.290
13047,2010-12-15,17.0,83.290
17850,2010-12-15,148.5,588.115
